# Chapter 5 - Resampling Methods: Applied Exercises

In [135]:
# import packages
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [136]:
def error_rate(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    correctly_classified = cm[0][0] + cm[1][1]
    total = np.sum(cm)
    percentage = correctly_classified/total*100
    error_rate = round(100 - percentage, 4)

    print('Error rate in percent:\t' + str(error_rate) +'%')

### Exercise 5

In [137]:
default = pd.read_csv('data/Default.csv')
default.head() # remove unnamed column

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.625074
1,2,No,Yes,817.180407,12106.134700
2,3,No,No,1073.549164,31767.138947
3,4,No,No,529.250605,35704.493935
4,5,No,No,785.655883,38463.495879


In [138]:
default.default.replace(('No', 'Yes'), (0,1), inplace=True)
default.student.replace(('No', 'Yes'), (0,1), inplace=True)
default = default.iloc[:, 1:]
default.head()

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


In [181]:
# Number of categroies
default['default'].nunique()

# Count yes and no for default
default.groupby('default').count()

# more compact counts
default.default.value_counts()

0    9667
1     333
Name: default, dtype: int64

In [168]:
# a) fit logReg -> default ~ income + balance
default_X = default[['income', 'balance']]
default_y = default['default']

logReg_full = LogisticRegression(penalty='none').fit(default_X, default_y)

default_y_predict = logReg_full.predict(default_X)
error_rate(default_y, default_y_predict)

[[9629   38]
 [ 225  108]]
Error rate in percent:	2.63%


In [171]:
# b) estimate test error
# i) train / test split
X_train, X_test, y_train, y_test =  train_test_split(default_X, default_y, test_size=0.5, random_state=42)

# ii) Fit multiple logReg
logReg_val = LogisticRegression(penalty='none').fit(X_train, y_train)

# iii) obtain a prediction of default status for each individual in the validation set
y_predict_val = logReg_val.predict(X_test)

# iv) compute the validation set error
error_rate(y_test, y_predict_val)

[[4839    2]
 [ 159    0]]
Error rate in percent:	3.22%


In [173]:
# c) repeat b) three times using three different splits
test_sizes = [0.33, 0.4, 0.6]

for test_size in test_sizes:
    X_train, X_test, y_train, y_test =  train_test_split(default_X, default_y, test_size=test_size, random_state=420)
    logReg_val = LogisticRegression(penalty='none').fit(X_train, y_train)
    y_predict_val = logReg_val.predict(X_test)
    print(f'Test size: {test_size}')
    error_rate(y_test, y_predict_val)

Test size: 0.33
[[3179   13]
 [  78   30]]
Error rate in percent:	2.7576%
Test size: 0.4
[[3870    2]
 [ 128    0]]
Error rate in percent:	3.25%
Test size: 0.6
[[5803    1]
 [ 196    0]]
Error rate in percent:	3.2833%


In [182]:
# d) fit logReg -> default ~ income + balance + student(0/1) and estimate test error
default_X = default[['income', 'balance', 'student']]

test_sizes = [0.33, 0.4, 0.5, 0.6]

for test_size in test_sizes:
    X_train, X_test, y_train, y_test =  train_test_split(default_X, default_y, test_size=test_size, random_state=420)
    logReg_val = LogisticRegression(penalty='none').fit(X_train, y_train)
    y_predict_val = logReg_val.predict(X_test)
    print(f'Test size: {test_size}')
    error_rate(y_test, y_predict_val)

Test size: 0.33
[[3165   27]
 [  83   25]]
Error rate in percent:	3.3333%
Test size: 0.4
[[3870    2]
 [ 128    0]]
Error rate in percent:	3.25%
Test size: 0.5
[[4796   40]
 [ 125   39]]
Error rate in percent:	3.3%
Test size: 0.6
[[5766   38]
 [ 153   43]]
Error rate in percent:	3.1833%


### Exercise 6

In [184]:
# a) determine estimated SE for coefficients (logReg -> default ~ income + balance)
import statsmodels.api as sm

default_X = default[['income', 'balance']]
X2 = sm.add_constant(default_X)

logReg2 = sm.Logit(default_y, X2).fit()
print(logReg2.summary())

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Fri, 15 Oct 2021   Pseudo R-squ.:                  0.4594
Time:                        11:32:16   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99

In [196]:
# b) write boot.fn() function that outputs the coefficient estimates for income and balance
def boot_fn(X, y, index_lower, index_upper):
    X = sm.add_constant(X).iloc[index_lower:index_upper, :]
    y = y.iloc[index_lower:index_upper]
    model = sm.Logit(y, X).fit()

    return model.params

coef = boot_fn(default_X, default_y, 0, 500)
print(coef)


Optimization terminated successfully.
         Current function value: 0.079979
         Iterations 10
const     -11.678916
income      0.000025
balance     0.005774
dtype: float64


In [211]:
# c) use bootstrap to estimate the SEs of the logReg coefficients 
from sklearn.utils import resample


def bootstrap_sample(data):
    boots_dataset = resample(data, replace=True, n_samples=len(data))
    return boots_dataset


def boot_fn(X, y):
    X = sm.add_constant(X)
    model = sm.Logit(y, X).fit(disp=0)

    return model.params


def boots_se(alpha):
    alpha_mean = np.mean(alpha)
    alpha_tilde = 0

    for a in alpha:
        alpha_tilde += (a - alpha_mean)**2

    return np.sqrt(1/(len(alpha)-1) * alpha_tilde)


const = []
income = []
balance = []


for b in range(0,100):
    boots_set = bootstrap_sample(default)
    coef = boot_fn(boots_set[['income', 'balance']], boots_set['default'])

    const.append(coef[0])
    income.append(coef[1])
    balance.append(coef[2])

print('Mean values of const, income, balance')
print(np.mean(const), np.mean(income), np.mean(balance))
print()
print('Std errors of const, income, balance')
print(boots_se(const), boots_se(income), boots_se(balance))

Mean values of const, income, balance
-11.551468290392854 2.0458080798937636e-05 0.005651696822938356

Std errors of const, income, balance
0.4177022491730254 4.9688417441790115e-06 0.0002100551679004928


In [146]:
# d) comment on the estimated standard errors obtained

Passt scho - stimmen überein.

### Exercise 7

In [147]:
weekly = pd.read_csv('data/Weekly.csv')
weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [148]:
# a) fit logReg -> Direction ~ Lag1 + Lag2

In [149]:
# b) fit logReg -> Direction ~ Lag1 + Lag2 with all but the first observation 

In [150]:
# c) use the model from b= to predict the direction of the first observation; was it classified correctly?

In [151]:
# d) for loop i-n:
# i) fit a logReg 
# ii) compute the posterior probability of the market moving up for the ith observation
# iii) use the posterior probability for the ith observation in order to predict whether or not the market moves up
# iv) Determine whether or not an error was made in predicting the direction for the ith observation (1 = error, 0 = none)

In [152]:
# e) take the avg of the n numbers obtained in d) in order to obtain the LOOCV estimate for the test error

### Exercise 8

In [153]:
# a) generate simulated data

In [154]:
# b) scatterplot of X against Y

In [155]:
# c) set a random seed and compute the LOOCV errors
# i), ii), iii), iv)

In [156]:
# d) repeat c) using a different seed

In [157]:
# e) which of the models in c) had the smallest LOOCV error?

In [158]:
# f) comment on the statistical significance of the coefficients in c)

### Exercise 9

In [159]:
boston = pd.read_csv('data/Boston.csv')
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [160]:
# a) estimate population mean of medv

In [161]:
# b) provide an estimate of the standard error of a)

In [162]:
# c) estimate SE of a) using bootstrap

In [163]:
# d) provide a 95% CI based on c) and compare it to t-Test

In [164]:
# e) provide an estimate for the median of medv

In [165]:
# f) estimate SE of median e) using bootstrap

In [166]:
# g) provide an estimate for tenth percentile of medv

In [167]:
# h) use bootstrap to estimate SE of g)